1. Seitenaufbau analysieren
2. Scrapen
3. Bereinigen
4. Analysieren
5. Visualisieren

# 1 Scrape/load data

In [1]:
# Load Data
import create_record
import pandas as pd

t1s114a1 = create_record.collect_data(1,108,1)
t1s114a1["Text"] = " ".join([i for i in [i.get_text() for i in t1s114a1["SoupContent"]]])

madhahib_df = pd.read_csv("./data/madhahib.csv", sep="\t", index_col=0)
tafasir_df = pd.read_csv("./data/tafasir.csv", sep="\t", index_col=0)


In [ ]:
##########################################################
# offline workaround
import pandas as pd
t1s114a2 = pd.read_csv('C:/Users/anaconda/Desktop/altafsir_scraper/corpus/1_1_7.csv', sep=",", index_col=0).T
t1s114a2

t1s114a1 = {}
t1s114a1["Text"] = t1s114a2["Text"][0]
###########################################################

____
# 2 Preprocessing
## 2.1 Normalizing data

In [2]:
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar


# def remove_citations(text):
#     import re
#     text_without_citations = re.sub("([\[\{]).*?([\]\}])", "", text)
    
#     return text_without_citations

def reduce_to_archarset(text):
    import re
    # Remove non-arabic characters
    nonarab_chars = '[^\u0621-\u064A ]'
    text = re.sub(nonarab_chars, '', text)
    return text

## normalize
def normalizer(string):
    string = normalize_unicode(string)
    string_normalized = normalize_alef_ar(string)
    string_normalized = normalize_alef_maksura_ar(string_normalized)
    string_normalized = normalize_teh_marbuta_ar(string_normalized)
    
    # remove diacritica
    string_normalized = dediac_ar(string_normalized)
    
    #reduce to arabic charset
    string_normalized = reduce_to_archarset(string_normalized)
    return string_normalized

t1s114a1["TextNormalized"] = normalizer(t1s114a1["Text"])

## 2.2 Tokenization

### 2.2.1 Simple Tokenization
Will tokenize words by splitting the string on whitespace and punctuation.

In [3]:
from camel_tools.tokenizers.word import simple_word_tokenize

t1s114a1["Tokenized"] = simple_word_tokenize(t1s114a1["TextNormalized"])

### 2.2.2 Morphological tokenization
The morphological tokenizer expects pre-tokenized text in a list. Therefore run simple_word_tokenize(string) first

In [5]:
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer


# Load a pretrained disambiguator to use with a tokenizer
mle = MLEDisambiguator.pretrained('calima-msa-r13')

# `split=True`: morphological tokens are output as seperate strings.
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)
t1s114a1["TokenizedMorph"] = tokenizer.tokenize(t1s114a1["Tokenized"])

# Rausfiltern unselbständiger Morpheme
t1s114a1["TokenizedMorph"] = [token for token in t1s114a1["TokenizedMorph"] if not '+' in token]

In [6]:
t1s114a1["TokenizedMorph"]

['يقول',
 'تعالى',
 'ذكر',
 'أنا',
 'أعطينا',
 'يا',
 'محمد',
 'كوثر',
 'أختلف',
 'أهل',
 'تأويل',
 'في',
 'معنى',
 'كوثر',
 'قال',
 'عض',
 'هو',
 'نهر',
 'في',
 'جنة',
 'أعطى',
 'الله',
 'نبيه',
 'محمدا',
 'صلى',
 'الله',
 'على',
 'سلم',
 'حدث',
 'يعقوب',
 'قال',
 'ثنا',
 'هشيم',
 'قال',
 'أخبر',
 'عطاء',
 'بن',
 'سائب',
 'عن',
 'محارب',
 'بن',
 'دثار',
 'عن',
 'ابن',
 'عمر',
 'أن',
 'قال',
 'كوثر',
 'نهر',
 'في',
 'جنة',
 'حافتا',
 'من',
 'ذهب',
 'فضة',
 'يجري',
 'على',
 'در',
 'ياقوت',
 'ماء',
 'أشد',
 'بياضا',
 'من',
 'لبن',
 'أحلى',
 'من',
 'عسل',
 'حدث',
 'ابن',
 'حميد',
 'قال',
 'ثنا',
 'جرير',
 'عن',
 'عطاء',
 'عن',
 'محارب',
 'بن',
 'دثار',
 'الباهلي',
 'عن',
 'ابن',
 'عمر',
 'في',
 'قول',
 'أنا',
 'أعطينا',
 'كوثر',
 'قال',
 'نهر',
 'في',
 'جنة',
 'حافتا',
 'ذهب',
 'مجرى',
 'على',
 'در',
 'ياقوت',
 'ماء',
 'أشد',
 'بياضا',
 'من',
 'ثلج',
 'أشد',
 'حلاوة',
 'من',
 'عسل',
 'تربة',
 'أطيب',
 'من',
 'ريح',
 'ألمس',
 'حدث',
 'أبو',
 'ريب',
 'قال',
 'ثنا',
 'عمر',
 'بن',
 'عبيد',
 

## [2.3 Remove stopwords from token list]
normalization needs to implemented for stoplist, before usable

In [18]:
def stopwords(token_list, stopwords_list, morph=True):

    token_list_filtered = []
    words_removed = []
    with open (stopwords_list, encoding="UTF-8") as stopwords:
        if morph == True:
            x = stopwords.read()
        else:
            x = normalizer(stopwords.read())
        for token in token_list:
            if token not in x:
                token_list_filtered.append(token)
            if token in x:
                words_removed.append(token)

    return token_list_filtered, words_removed

In [19]:
t1s114a1["TokenizedMorphStopword"] = stopwords(t1s114a1["TokenizedMorph"], 'C:/Users/anaconda/Desktop/arabic-stop-words-master/list2.txt')[0]
t1s114a1["TokenizedStopword"] = stopwords(t1s114a1["Tokenized"], 'C:/Users/anaconda/Desktop/arabic-stop-words-master/list2.txt', morph=False)[0]


----
# 3 Analyzing data
## 3.1 Morphological analysis of tokens

In [7]:
import pandas as pd
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer


# set up morphological analyzer
db = MorphologyDB.builtin_db("calima-msa-r13")
analyzer = Analyzer(db)

# create dictionary for every token taking index value in t1s114a1["tokenized"] as key
t1s114a1_token_analysis = {}
for i, val in enumerate(t1s114a1["Tokenized"]):
    t1s114a1_token_analysis[i] = pd.DataFrame(analyzer.analyze(t1s114a1["Tokenized"][i]))


In [12]:
t1s114a1_token_analysis[2]


,diac,lex,bw,gloss,pos,prc3,prc2,prc1,prc0,per,...,root,bwtok,pattern,lex_logprob,atbtok,atbseg,d1seg,stem,stemgloss,stemcat
0,ذَكَرَهُ,ذَكَر,ذَكَر/PV+َ/PVSUFF_SUBJ:3MS+هُ/PVSUFF_DO:3MS,mention;cite;remember+he;it_<verb>_it;him,verb,0,0,0,0,3,...,ذ.ك.ر,ذَكَر_+َ_+هُ,1َ2َ3َهُ,-3.243699,ذَكَرَ_+هُ,ذَكَرَ_+هُ,ذَكَرَهُ,ذَكَر,mention;cite;remember,PV
1,ذَكَّرَهُ,ذَكَّر,ذَكَّر/PV+َ/PVSUFF_SUBJ:3MS+هُ/PVSUFF_DO:3MS,remind+he;it_<verb>_it;him,verb,0,0,0,0,3,...,ذ.ك.ر,ذَكَّر_+َ_+هُ,1َ2َّ3َهُ,-4.068112,ذَكَّرَ_+هُ,ذَكَّرَ_+هُ,ذَكَّرَهُ,ذَكَّر,remind,PV
2,ذِكْرُهُ,ذِكْر,ذِكْر/NOUN+ُ/CASE_DEF_NOM+هُ/POSS_PRON_3MS,mention;citation;memory+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذِكْر_+ُ_+هُ,1ِ2ْ3ُهُ,-4.339853,ذِكْرُ_+هُ,ذِكْرُ_+هُ,ذِكْرُهُ,ذِكْر,mention;citation;memory,N
3,ذِكْره,ذِكْر,ذِكْر/NOUN+هُ/POSS_PRON_3MS,mention;citation;memory+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذِكْر_+هُ,1ِ2ْ3ه,-4.339853,ذِكْر_+هُ,ذِكْر_+هُ,ذِكْرهُ,ذِكْر,mention;citation;memory,N
4,ذِكْرَهُ,ذِكْر,ذِكْر/NOUN+َ/CASE_DEF_ACC+هُ/POSS_PRON_3MS,mention;citation;memory+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذِكْر_+َ_+هُ,1ِ2ْ3َهُ,-4.339853,ذِكْرَ_+هُ,ذِكْرَ_+هُ,ذِكْرَهُ,ذِكْر,mention;citation;memory,N
5,ذِكْرِهِ,ذِكْر,ذِكْر/NOUN+ِ/CASE_DEF_GEN+هُ/POSS_PRON_3MS,mention;citation;memory+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذِكْر_+ِ_+هِ,1ِ2ْ3ِهِ,-4.339853,ذِكْرِ_+هُ,ذِكْرِ_+هِ,ذِكْرِهِ,ذِكْر,mention;citation;memory,N
6,ذَكَرُهُ,ذَكَر,ذَكَر/NOUN+ُ/CASE_DEF_NOM+هُ/POSS_PRON_3MS,male+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذَكَر_+ُ_+هُ,1َ2َ3ُهُ,-4.286608,ذَكَرُ_+هُ,ذَكَرُ_+هُ,ذَكَرُهُ,ذَكَر,male,N
7,ذَكَره,ذَكَر,ذَكَر/NOUN+هُ/POSS_PRON_3MS,male+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذَكَر_+هُ,1َ2َ3ه,-4.286608,ذَكَر_+هُ,ذَكَر_+هُ,ذَكَرهُ,ذَكَر,male,N
8,ذَكَرَهُ,ذَكَر,ذَكَر/NOUN+َ/CASE_DEF_ACC+هُ/POSS_PRON_3MS,male+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذَكَر_+َ_+هُ,1َ2َ3َهُ,-4.286608,ذَكَرَ_+هُ,ذَكَرَ_+هُ,ذَكَرَهُ,ذَكَر,male,N
9,ذَكَرِهِ,ذَكَر,ذَكَر/NOUN+ِ/CASE_DEF_GEN+هُ/POSS_PRON_3MS,male+its;his,noun,0,0,0,0,na,...,ذ.ك.ر,ذَكَر_+ِ_+هِ,1َ2َ3ِهِ,-4.286608,ذَكَرِ_+هُ,ذَكَرِ_+هِ,ذَكَرِهِ,ذَكَر,male,N


## 3.2 Frequency analysis

In [13]:
# create with root as key and frequency count as val
def frequency_analyzer(token_list):
    token_freqs = {}
    for token in token_list:
        if token in token_freqs:
            token_freqs[token] += 1
        else:
            token_freqs[token] = 1

    # sorting
    token_freqs = {k: v for k, v in sorted(token_freqs.items(), key=lambda item: item[1], reverse=True)}
    return token_freqs

### 3.2.1 ... of morphologically tokenized tokens

In [22]:
frequency_analyzer(t1s114a1["TokenizedMorphStopword"])
# frequency_analyzer(t1s114a1["TokenizedStopword"])

{'الله': 106,
 'حدث': 89,
 'ابن': 83,
 'كوثر': 56,
 'سلم': 41,
 'أنحر': 39,
 'صلى': 37,
 'جنة': 36,
 'نهر': 34,
 'فصل': 34,
 'قول': 33,
 'صلاة': 26,
 'عبد': 24,
 'رسول': 24,
 'أبتر': 24,
 'محمد': 22,
 'سعيد': 22,
 'نحر': 21,
 'ذكر': 19,
 'حميد': 19,
 'ثنى': 17,
 'شانئ': 17,
 'أعطينا': 15,
 'عطاء': 15,
 'جبير': 15,
 'أنس': 15,
 'نبي': 15,
 'وكيع': 14,
 'عباس': 13,
 'سفيان': 13,
 'يقول': 12,
 'ريب': 12,
 'حافتا': 11,
 'عكرمة': 11,
 'عاصم': 11,
 'قتاد': 11,
 'بدن': 11,
 'قلت': 10,
 'جعفر': 10,
 'مهران': 10,
 'آخرون': 10,
 'ياقوت': 9,
 'لؤلؤ': 9,
 'نجيح': 9,
 'مجاهد': 9,
 'يزيد': 9,
 'بشار': 9,
 'عقبة': 9,
 'سائب': 8,
 'وضع': 8,
 'يعقوب': 7,
 'عسل': 7,
 'عائشة': 7,
 'سريج': 7,
 'رحمن': 7,
 'العاص': 7,
 'وائل': 7,
 'ماء': 6,
 'أحلى': 6,
 'عبيد': 6,
 'أحمد': 6,
 'مجوف': 6,
 'جبريل': 6,
 'أعناق': 6,
 'نسك': 6,
 'آية': 6,
 'محارب': 5,
 'دثار': 5,
 'أشد': 5,
 'الرازي': 5,
 'سألت': 5,
 'مثني': 5,
 'عمارة': 5,
 'حسن': 5,
 'عثمان': 5,
 'أعطي': 5,
 'أقوال': 5,
 'يونس': 5,
 'نزلت': 5,
 'مكتوبة': 5,


### 3.2.2 ... of roots

In [23]:
# create list of roots for the lemma with max probability for a given token from tokens_analysis
t1s114a1["RootsList"] = []
for i in range(1, len(t1s114a1["Tokenized"])):
    if "lex_logprob" in t1s114a1_token_analysis[i]:
        t1s114a1["RootsList"].append(t1s114a1_token_analysis[i][t1s114a1_token_analysis[i].lex_logprob == t1s114a1_token_analysis[i].lex_logprob.max()].iloc[0]["root"])


In [24]:
frequency_analyzer(t1s114a1["RootsList"])

{'ق.#.ل': 399,
 'ع.ن': 220,
 'ب.ن.#': 208,
 '#.ل.ه': 107,
 '#.ب.#': 107,
 'ح.د.ث': 92,
 '#.ن.ن': 89,
 'ع.ل.#': 85,
 'ف.#': 78,
 'ك.ث.ر': 75,
 'م.ن': 70,
 'ن.ح.ر': 69,
 'ص.ل.#': 68,
 'ه': 67,
 'ع.ط.#': 53,
 'ح.م.د': 51,
 'س.ل.م': 47,
 'ر.ب.ب': 38,
 'ن.ه.ر': 36,
 'ج.ن.ن': 36,
 'ذ.#': 34,
 'ف.ص.ل': 34,
 'NTWS': 33,
 'ع.ب.د': 31,
 'س.ع.د': 29,
 'خ.#.ر': 29,
 '#.ل': 27,
 'ب.ت.ر': 27,
 'م.#': 25,
 'ث.ن.#': 24,
 'ر.س.ل': 24,
 'ل': 22,
 'خ.ب.ر': 21,
 'ع.م.ر': 20,
 'ذ.ك.ر': 19,
 'ع.ق.ب': 19,
 'ج.ب.ر': 19,
 '#.#.ل': 18,
 'ن.ب.#': 18,
 'ش.ن.#': 17,
 '#.#.#': 16,
 '#.#': 15,
 'ن.س.#': 15,
 'ز.#.د': 15,
 'ك.#.ن': 15,
 'ع.ب.س': 13,
 'ن.ع.م': 13,
 'س.ف.#': 13,
 'ب.ع.ض': 12,
 'ر.#.ب': 12,
 '#.د.#': 12,
 '#.ل.#': 12,
 'ح.#.ف': 11,
 '#.خ.ر': 11,
 'عكرمة': 11,
 'ع.ص.م': 11,
 'ق.ت.د': 11,
 'ن.ز.ل': 11,
 'ب.د.ن': 11,
 '#.ه.ل': 10,
 '#.ق.ت': 10,
 'ج.ع.ف.ر': 10,
 'م.ه.ر': 10,
 'ع.ن.#': 9,
 'ل.#.ل.#': 9,
 'ن.ج.ح': 9,
 'ج.ه.د': 9,
 'س.م.ع': 9,
 'ش.ر.ر': 9,
 'ب.ش.ر': 9,
 'غ.#.ر': 9,
 'س.#.ب': 8,
 'د.ر.ر': 8,
 '

## 3.3 Named-entity recognition
Hint: it seems that simple tokenization is more apt for NER, as it doesn't remove enclitics like "ك" from parts of the name

In [25]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.ner import NERecognizer

ner = NERecognizer.pretrained()

# NERecognizer expects pre-tokenized text
sentence = t1s114a1["Tokenized"] # simple_word_tokenize(t1s114a1["Text"])

labels = ner.predict_sentence(sentence)

# save each token paired with it's NER label
zipped = list(zip(sentence, labels))


In [29]:
# Filter and glue named entities into dictionary
named_entities = {"LOC" : [], "ORG" : [], "PERS" : [], "MISC" : []}
for i, val in enumerate(zipped):
    if zipped[i][1][0] == "B":
        named_entities[zipped[i][1][2:]].append(zipped[i][0])
    if zipped[i][1][0] == "I":
        named_entities[zipped[i][1][2:]][-1] = named_entities[zipped[i][1][2:]][-1] + " " + zipped[i][0]
named_entities["MISC"]

['الله',
 'الله',
 'الكوثر',
 'الكوثر',
 'الياقوت',
 'اللؤلؤ',
 'الكوثر',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الياقوت',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'اللؤلؤ',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الياقوت',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الله',
 'الكوثر',
 'الله',
 'الله',
 'الله',
 'الله',
 'الفجر',
 'الله',
 'الله',
 'الله',
 'لله',
 'بالله',
 'الله',
 'الله',
 'الحديبيه',
 'الله',
 'البيت',
 'الله',
 'الله',
 'الله',
 'ربك',
 'الله',
 'الله',
 'الكوثر',
 'الابتر',
 'ال